In [16]:
from yahooquery import Ticker
from openai import OpenAI
from dotenv import load_dotenv
import os
import math 
from scipy.stats import norm
load_dotenv()
import pandas as pd

In [ ]:
from scipy.stats import norm
from scipy.optimize import brentq
import math

def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    if T <= 0 or sigma <= 0:
        return max(0.0, S - K) if option_type == 'call' else max(0.0, K - S)

    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return S * norm.pdf(d1) * math.sqrt(T)

def implied_volatility(option_price, S, K, T, r, option_type='call'):
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - option_price

    try:
        # Safe bounded root-finding
        return brentq(objective, 1e-6, 3.0, maxiter=1000)
    except (ValueError, RuntimeError):
        return float('nan')

In [ ]:
stocks = ['UNH']
unh = Ticker(stocks)

df = unh.option_chain
df.head()

contractSymbol  strike currency  lastPrice  \
symbol expiration optionType                                                   
UNH    2025-05-23 calls       UNH250523C00170000   170.0      USD     118.00   
                  calls       UNH250523C00200000   200.0      USD      90.50   
                  calls       UNH250523C00210000   210.0      USD      84.40   
                  calls       UNH250523C00220000   220.0      USD      72.80   
                  calls       UNH250523C00222500   222.5      USD      67.41   

                              change  volume     bid     ask contractSize  \
symbol expiration optionType                                                
UNH    2025-05-23 calls       118.00    20.0  117.20  123.50      REGULAR   
                  calls        90.50    46.0   88.10   94.65      REGULAR   
                  calls        84.40     7.0   78.75   85.65      REGULAR   
                  calls        72.80     8.0   69.75   74.45      REGULAR   
                  calls        67.41     4.0   66.55   71.80      REGULAR   

                                   lastTradeDate  impliedVolatility  \
symbol expiration optionType                                          
UNH    2025-05-23 calls      2025-05-16 17:12:46           2.588871   
                  calls      2025-05-16 19:44:30           2.182622   
                  calls      2025-05-16 19:15:55           1.250004   
                  calls      2025-05-16 19:11:55           1.027349   
                  calls      2025-05-16 17:09:07           1.624514   

                              inTheMoney  openInterest  percentChange  
symbol expiration optionType                                           
UNH    2025-05-23 calls             True           0.0            0.0  
                  calls             True          70.0            0.0  
                  calls             True          11.0            0.0  
                  calls             True          23.0            0.0  
                  calls             True           0.0            0.0

In [23]:
price = unh.financial_data['UNH']['currentPrice']
print(price)
for index, row in df[:5].iterrows():
    print(row)
    option_price = row['lastPrice']
    strike_price = row['strike']
    expiry = index[1]
    now = pd.Timestamp.now()
    time_to_expiration = (expiry - now).total_seconds() / (365.25 * 24 * 60 * 60)
    risk_free_rate =  0.05
    print(time_to_expiration)

    print(f"option price: {option_price}")
    print(f"price: {price}")
    print(f"strike_price: {strike_price}")
    print(f"time to expiration; {time_to_expiration}")
    print(f"risk free rate: {risk_free_rate}")


    iv = implied_volatility(option_price, price, strike_price, 
                            time_to_expiration, risk_free_rate,
                            option_type='call'
                            )
    print(iv)
    print(row['impliedVolatility'])

   


291.91
contractSymbol        UNH250523C00170000
strike                             170.0
currency                             USD
lastPrice                          118.0
change                             118.0
volume                              20.0
bid                                117.2
ask                                123.5
contractSize                     REGULAR
lastTradeDate        2025-05-16 17:12:46
impliedVolatility               2.588871
inTheMoney                          True
openInterest                         0.0
percentChange                        0.0
Name: (UNH, 2025-05-23 00:00:00, calls), dtype: object
0.011448307516667934
option price: 118.0
price: 291.91
strike_price: 170.0
time to expiration; 0.011448307516667934
risk free rate: 0.05
-3.342945646907077e+138
inf
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

/tmp/ipykernel_18347/3646165150.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  sigma -= price_diff / v
/tmp/ipykernel_18347/3646165150.py:3: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
/tmp/ipykernel_18347/3646165150.py:17: RuntimeWarning: invalid value encountered in scalar divide
  d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))


RuntimeError: Implied volatility did not converge after 100 iterations

In [4]:
df.shape

(2426, 14)

In [5]:
df.to_csv("unh_options.csv")

In [ ]:
X_API_KEY = os.getenv("X_API_KEY")
client = OpenAI(
    api_key = X_API_KEY,
    base_url ="https://api.x.ai/v1"
)

completion = client.chat.completions.create(
    model="grok-3-mini-beta",
    messages = [
        {'role':'user', 'content':'should i buy unh stock options?'}
    ]
)

print(completion)